In [2]:
import numpy as np
import pandas as pd

from tqdm import tqdm, tqdm_notebook

from scipy import stats
from sklearn.model_selection import GroupKFold

import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.utils.data
#from transformers import *
import transformers
from transformers import AutoTokenizer, AutoModel, AutoConfig
import os
import re
import math
import random
from matplotlib import pyplot as plt
import warnings
from math import floor, ceil

warnings.filterwarnings('ignore')
device = torch.device('cuda')
torch.backends.cudnn.benchmark=True

%matplotlib inline

In [3]:
train = pd.read_csv('C:/Users/Lab000/Desktop/kaggle/kaggle_competetion/Google_Quest_LABEL/my-solution/input/google-quest-challenge/train.csv').fillna(' ')
test = pd.read_csv('C:/Users/Lab000/Desktop/kaggle/kaggle_competetion/Google_Quest_LABEL/my-solution/input/google-quest-challenge/test.csv').fillna(' ')
sub = pd.read_csv('C:/Users/Lab000/Desktop/kaggle/kaggle_competetion/Google_Quest_LABEL/my-solution/input/google-quest-challenge/sample_submission.csv').fillna(' ')

In [4]:
train.columns

Index(['qa_id', 'question_title', 'question_body', 'question_user_name',
       'question_user_page', 'answer', 'answer_user_name', 'answer_user_page',
       'url', 'category', 'host', 'question_asker_intent_understanding',
       'question_body_critical', 'question_conversational',
       'question_expect_short_answer', 'question_fact_seeking',
       'question_has_commonly_accepted_answer',
       'question_interestingness_others', 'question_interestingness_self',
       'question_multi_intent', 'question_not_really_a_question',
       'question_opinion_seeking', 'question_type_choice',
       'question_type_compare', 'question_type_consequence',
       'question_type_definition', 'question_type_entity',
       'question_type_instructions', 'question_type_procedure',
       'question_type_reason_explanation', 'question_type_spelling',
       'question_well_written', 'answer_helpful',
       'answer_level_of_information', 'answer_plausible', 'answer_relevance',
       'answer_satisfa

In [5]:
def _get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    first_sep = True
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            if first_sep:
                first_sep = False
                current_segment_id = 1#新增 
            else:
                current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

def _get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

def _trim_input(title, question, answer, max_sequence_length=512-1, 
                t_max_len=70-1, q_max_len=219, a_max_len=219):#???

    t = tokenizer.tokenize(title)
    q = tokenizer.tokenize(question)
    a = tokenizer.tokenize(answer)
    
    t_len = len(t)
    q_len = len(q)
    a_len = len(a)

    if (t_len+q_len+a_len+4) > max_sequence_length:
        
        if t_max_len > t_len:
            t_new_len = t_len
            a_max_len = a_max_len + floor((t_max_len - t_len)/2)
            q_max_len = q_max_len + ceil((t_max_len - t_len)/2)
        else:
            t_new_len = t_max_len
      
        if a_max_len > a_len:
            a_new_len = a_len 
            q_new_len = q_max_len + (a_max_len - a_len)
        elif q_max_len > q_len:
            a_new_len = a_max_len + (q_max_len - q_len)
            q_new_len = q_len
        else:
            a_new_len = a_max_len
            q_new_len = q_max_len
            
            
        if t_new_len+a_new_len+q_new_len+4 != max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d" 
                             % (max_sequence_length, (t_new_len+a_new_len+q_new_len+4)))

        t = t[:t_new_len]
        q = norm_token_length(q, q_new_len)
        a = norm_token_length(a, a_new_len)
    
    return t, q, a

def norm_token_length(tokens, l):
    if len(tokens) > l:
        head = l//2
        tail = l - head
        return tokens[:head] + tokens[-tail:]
    else:
        return tokens[:l]

def _convert_to_bert_inputs(title, question, answer, cate, max_sequence_length=512):
    """Converts tokenized input to ids, masks and segments for BERT"""
    #stoken = ["[CLS]"] + [cate] + title + ["[SEP]"] + question + ["[SEP]"] + answer + ["[SEP]"]
    stoken_1 = ["[CLS]"] + [cate] + title + ["[SEP]"] + question + ["[SEP]"]
    stoken_2 = ["[CLS]"] + [cate]+title + ["[SEP]"] + answer + ["[SEP]"]
    input_ids = _get_ids(stoken_1, tokenizer, max_sequence_length)
    input_ids_2 = _get_ids(stoken_2, tokenizer, max_sequence_length)
    input_segments = _get_segments(stoken_1, max_sequence_length)
    input_segments_2 = _get_segments(stoken_2, max_sequence_length)
    
    #return [input_ids, input_segments]
    return input_ids, input_segments,input_ids_2,input_segments_2

def convert_row(row,pretrained_weights):
    #c = f"[{row['category'].lower()}]"

    if pretrained_weights == "bert-base-uncased":
        c = f"[{row['category'].lower()}]"#type:str
    elif pretrained_weights == "bert-base-cased":
        c = f"[{row['category']}]"#type:str
    elif pretrained_weights == "xlnet-base-cased":
        c = f"[{row['category']}]"#type:str
    t, q, a = title = row["question_title"], row["question_body"], row["answer"]#type:str


    t, q, a = row["question_title"], row["question_body"], row["answer"]
    t, q, a = _trim_input(t, q, a)
    #ids, segments = _convert_to_bert_inputs(t, q, a, c)
    ids, segments, ids2, segments2 = _convert_to_bert_inputs(t, q, a, c)
    #total_input=[np.array([[ids, segments]]),np.array([[ids2, segments2]])]
    # total_input=[]
    # print(np.array([[ids, segments]]).shape)
    # total_input.append(np.array([[ids, segments]]))
    # total_input.append(np.array([[ids2, segments2]]))
    # return total_input
    return np.array([[ids, segments, ids2, segments2]])

In [6]:
#model="bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained("./double/token_model_config/tokenizer/")
#tokenizer.add_tokens("./bert_based_tokenizer/added_tokens.json")
#tokenizer.add_tokens(categories)


In [7]:
len(tokenizer)

29001

In [8]:
def custom_loss(data,data2, targets,targets2):
    
    # mse = nn.MSELoss(reduction="none")(data[:,:30].sigmoid(), targets[:,:30])
    # bce = nn.BCEWithLogitsLoss(reduction='none')(data[:,:30], targets[:,:30])#??
    mse = nn.MSELoss(reduction="none")(data[:,:].sigmoid(), targets[:,:])
    bce = nn.BCEWithLogitsLoss(reduction='none')(data[:,:], targets[:,:])#??
    
    mse2 = nn.MSELoss(reduction="none")(data2[:,:].sigmoid(), targets2[:,:])
    bce2 = nn.BCEWithLogitsLoss(reduction='none')(data2[:,:], targets2[:,:])#??
    #w =  targets[:,30:]
    #loss = (mse*w).sum() + bce.sum()
    loss = (mse).sum()+ bce.sum()+mse2.sum()+bce2.sum()
    return loss

class CustomBert(nn.Module):
    def __init__(self, config_path=None):
        super(CustomBert, self).__init__()
        self.config = AutoConfig.from_pretrained(config_path) 
        
        self.config.Q_labels = 21
        self.config.A_labels = 9
        self.config.output_hidden_states = True
        self.n_use_layer = 4 #原本
        #self.n_use_layer = 2
        self.double_bert= 1
        self.n_labels = self.config.num_labels
        #self.config.save_pretrained("bert_based_config")
        #self.config.save_pretrained(output_dir+"config")
        #self.bert = BertModel(config)
        self.bert=AutoModel.from_config(self.config)
        self.bert2=AutoModel.from_config(self.config)
        #self.bert.save_pretrained('bert_based_model')
        #self.bert.save_pretrained(output_dir+"model")
        # self.dense1 = nn.Linear(self.config.hidden_size*self.n_use_layer, self.config.hidden_size*self.n_use_layer)
        # self.dense2 = nn.Linear(self.config.hidden_size*self.n_use_layer, self.config.hidden_size*self.n_use_layer)
        # self.dropout = nn.Dropout(self.config.hidden_dropout_prob)
        # self.classifier = nn.Linear(self.config.hidden_size*self.n_use_layer, self.config.num_labels)

        self.dense1 = nn.Linear(self.double_bert*self.config.hidden_size*self.n_use_layer, self.double_bert*self.config.hidden_size*self.n_use_layer)
        self.dense2 = nn.Linear(self.double_bert*self.config.hidden_size*self.n_use_layer, self.double_bert*self.config.hidden_size*self.n_use_layer)
        self.dropout = nn.Dropout(self.config.hidden_dropout_prob)
        self.classifier = nn.Linear(self.double_bert*self.config.hidden_size*self.n_use_layer, self.config.Q_labels)
        self.classifier2 = nn.Linear(self.double_bert*self.config.hidden_size*self.n_use_layer, self.config.A_labels)
        #self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None
                ,input_ids2=None, attention_mask2=None, token_type_ids2=None,position_ids=None, head_mask=None, inputs_embeds=None, labels=None):

        # outputs = self.bert(input_ids,
        #                     attention_mask=attention_mask,
        #                     token_type_ids=token_type_ids,
        #                     position_ids=position_ids,
        #                     head_mask=head_mask,
        #                     inputs_embeds=inputs_embeds)
        outputs = self.bert(input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids
                            )
        outputs2 = self.bert2(input_ids2,
                            attention_mask=attention_mask2,
                            token_type_ids=token_type_ids2
                            )
        
        #print(outputs[2][-1].shape)
        pooled_output = torch.cat([outputs[2][-1*(i+1)][:,0] for i in range(self.n_use_layer)], dim=1)#把倒數最後4個layer的cls output concat在一起，把4個(8,768) concat，變成(8,3072) #原本
        pooled_output2 = torch.cat([outputs2[2][-1*(i+1)][:,0] for i in range(self.n_use_layer)], dim=1)
        #pooled_output = torch.cat([outputs[2][-1*(i+1)][:,0] for i in range(self.n_use_layer)], dim=1)#把倒數最後2個layer的cls output concat在一起,把2個(8,768) concat，變成(8,1536)
        #pooled_output2 = torch.cat([outputs2[2][-1*(i+1)][:,0] for i in range(self.n_use_layer)], dim=1)#同上
        #double_pooled_output=torch.cat([pooled_output,pooled_output],dim=1)#(8,3072)
        
        pooled_output = self.dense1(pooled_output)
        pooled_output = self.dense2(pooled_output)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        pooled_output2 = self.dense1(pooled_output2)
        pooled_output2 = self.dense2(pooled_output2)
        pooled_output2 = self.dropout(pooled_output2)
        logits2 = self.classifier2(pooled_output2)

        # double_pooled_output = self.dense1(double_pooled_output)
        # double_pooled_output = self.dense2(double_pooled_output)
        # double_pooled_output = self.dropout(double_pooled_output)
        # logits = self.classifier(double_pooled_output)

        outputs = (logits,) + outputs[2:]
        outputs2 = (logits2,) + outputs2[2:]

        return outputs,outputs2

In [9]:
model=CustomBert("double/token_model_config/config/config.json")


In [10]:
BS=8

In [11]:
pretrained_weights = 'bert-base-cased'
X_test = test[["question_title", "question_body", "answer", "category"]].apply(lambda x: convert_row(x, pretrained_weights), axis=1).values#shape(476)
#X_train = train[["question_title", "question_body", "answer", "category"]].apply(lambda x: convert_row(x, pretrained_weights), axis=1).values#shape(476)
#np.vstack(X_test).shape : (476, 2, 512)
X_test = np.vstack(X_test).reshape((len(X_test), 2048))

test_dataset = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long))
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BS, shuffle=False)

Token indices sequence length is longer than the specified maximum sequence length for this model (4324 > 512). Running this sequence through the model will result in indexing errors


In [12]:
model = model.to(device)
for param in model.parameters():
    param.requires_grad=False
model.bert.resize_token_embeddings(len(tokenizer))#??
model.bert2.resize_token_embeddings(len(tokenizer))
model.eval()

CustomBert(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29001, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (d

In [13]:
#model_dir_target = "../input/Optimize binning/bert-base-cased"#

cased_pred_lst = []
for fold in range(10):
    # if fold in [0,1,2,3,4,5,6,7]:
    #     continue
    #bert_path = f"{model_dir_target}/bert-base-cased_f{fold}_best"
    #bert_path=f"./DoubleBertBasedCase/bce_no_opt_binning/double-bert-based-case_f{fold}_best"
    bert_path=f"./double/Bce-NoOptbinning/double-bert-based-case_f{fold}_best"
    model.load_state_dict(torch.load(bert_path),strict=False)
    
    lst = []
    for i, (x_batch,)  in enumerate(test_loader):
        # input_ids = x_batch[:, :512]
        # token_ids = x_batch[:, 512:]
        input_ids = x_batch[:, :512]
        token_ids = x_batch[:, 512:1024]
        input_ids2 = x_batch[:, 1024:1536]
        token_ids2 = x_batch[:, 1536:]
        #pred = model(input_ids.to(device), attention_mask=(input_ids > 0).to(device), token_type_ids=token_ids.to(device))
        pred, pred2 = model(input_ids.to(device), attention_mask=(input_ids > 0).to(device), token_type_ids=token_ids.to(device),
                     input_ids2=input_ids2.to(device),attention_mask2=(input_ids2 > 0).to(device),token_type_ids2=token_ids2.to(device))
        total_y_pred=torch.cat((pred[0],pred2[0]),dim=1)
        lst.append(total_y_pred.detach().cpu().squeeze().numpy())
    train_pred = np.vstack(lst)#shape:(476, 30)
    
    cased_pred_lst.append(train_pred)

In [14]:
cased_pred_lst[0].shape

(476, 30)

In [15]:
cased_pred_lst

[array([[ 2.9148734 ,  0.5870639 , -1.0286921 , ..., -3.9417024 ,
          3.10868   ,  2.7866461 ],
        [ 1.9138436 , -0.14367273, -5.5822263 , ..., -2.0265653 ,
         -2.8845267 ,  2.1542585 ],
        [ 2.260144  ,  0.7972341 , -4.2451344 , ..., -3.3293252 ,
          2.7163298 ,  2.2225938 ],
        ...,
        [ 1.6526257 , -0.734839  , -4.6569324 , ..., -1.4699891 ,
         -0.48530427,  1.9625342 ],
        [ 3.188662  ,  1.6525309 , -4.248226  , ..., -2.487862  ,
          3.2956543 ,  2.9502594 ],
        [ 2.392885  ,  0.2380366 , -3.6111574 , ..., -1.605704  ,
         -1.4362035 ,  2.105583  ]], dtype=float32),
 array([[ 2.5103564 ,  0.45609975, -2.0337408 , ..., -3.6385996 ,
          2.5943546 ,  2.7311163 ],
        [ 1.9764313 , -0.17333022, -6.054926  , ..., -1.9884539 ,
         -2.7503853 ,  2.3761356 ],
        [ 2.3559232 ,  0.8823283 , -4.627333  , ..., -3.3722978 ,
          2.6931622 ,  2.3779893 ],
        ...,
        [ 1.5696981 , -0.64607036, -4.4

In [16]:
a=[[1,2,3],[4,5,6]]#(2,3)
#b=[[7,8,9],[10,11,12]]
a=np.array(a)
#b=np.array(b)
c=[]
c.append(a)
#c.append(b)
np.array(c).mean(0)

array([[1., 2., 3.],
       [4., 5., 6.]])

In [ ]:
np.array(cased_pred_lst).mean(0)

In [17]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))
cased_pred = np.array(cased_pred_lst).mean(0)
cased_pred = sigmoid(cased_pred)
cased_pred

array([[0.94259244, 0.6083406 , 0.1902955 , ..., 0.02070517, 0.9394443 ,
        0.92679155],
       [0.8596125 , 0.46842766, 0.00412077, ..., 0.11832326, 0.04951198,
        0.90402186],
       [0.9206932 , 0.6916602 , 0.01288849, ..., 0.03152938, 0.9470751 ,
        0.91943383],
       ...,
       [0.8398634 , 0.337506  , 0.01427344, ..., 0.16884752, 0.40362835,
        0.8926889 ],
       [0.9413441 , 0.83894837, 0.02454691, ..., 0.07342236, 0.95903397,
        0.94206464],
       [0.90788543, 0.5112956 , 0.01817529, ..., 0.12527874, 0.24396653,
        0.89683133]], dtype=float32)

#4TH Postprocessing

In [ ]:
import copy
from scipy import stats

# def spearmanr(y_true, y_pred):
#     if np.ndim(y_pred) == 2:
#         corr = np.nan_to_num([stats.spearmanr(y_true[:, i], y_pred[:, i])[0] for i in range(y_true.shape[1])]).mean()
#     else:
#         corr = stats.spearmanr(y_true, y_pred)[0]
#     return corr
def metric3(y_true,y_pred):
    y = copy.deepcopy(y_pred) #make_copy
    list_of_max_voters=[] #  list_of_max_voters[i] = how many voters did label the data (instead of 90 for all the columns)
    for i in (range(y_pred.shape[1])):
        best_score= 0 #initilize score for the the column i
        best_max_voters=1 #
        history_score=[]
        for max_voters in range(1,200):
            y[:,i]= (y_pred[:,i]//(1/max_voters))*(1/max_voters)
            score = stats.spearmanr(y_true[:, i], y[:, i]).correlation
            history_score.append(score)
            if score > best_score:
                best_score = score
                best_max_voters= max_voters
        list_of_max_voters.append(best_max_voters)

        y[:,i]= (y_pred[:,i]//(1/best_max_voters))*(1/best_max_voters)
    return np.mean([stats.spearmanr(y_true[:, ind], y[:, ind]).correlation for ind in range(y.shape[1])])#,list_of_max_voters```

In [ ]:
a=metric3(y3,cased_pred)

#1th PostProcessing

In [18]:
from collections import Counter

def postprocess_single(target, ref):
    """
    The idea here is to make the distribution of a particular predicted column
    to match the correspoding distribution of the corresponding column in the
    training dataset (called ref here)
    """
    
    ids = np.argsort(target)
    counts = sorted(Counter(ref).items(), key=lambda s: s[0])
    scores = np.zeros_like(target)
    
    last_pos = 0
    v = 0
    
    for value, count in counts:
        next_pos = last_pos + int(round(count / len(ref) * len(target)))
        if next_pos == last_pos:
            next_pos += 1

        cond = ids[last_pos:next_pos]
        scores[cond] = v
        last_pos = next_pos
        v += 1
        
    return scores / scores.max()

In [19]:
def postprocess_prediction(prediction, actual):
    
    postprocessed = prediction.copy()
    
    for col in target_columns:
        scores = postprocess_single(prediction[col].values, actual[col].values)
        # Those are columns where our postprocessing gave substantial improvement.
        # It also helped for some others, but we didn't include them as the gain was
        # very marginal (less than 0.01)
        if col in (
            "question_conversational",
            "question_type_compare",
            "question_type_definition",
            "question_type_entity",
            "question_has_commonly_accepted_answer",
            "question_type_consequence",
            "question_type_spelling"
        ):
            postprocessed[col] = scores
            
        # scale to 0-1 interval
        v = postprocessed[col].values
        postprocessed[col] = (v - v.min()) / (v.max() - v.min())
    
    return postprocessed

In [20]:
test_copy=test.copy()
target_columns=sub.columns
target_columns=target_columns[1:]
for id,target in enumerate(target_columns):
    test_copy[target]=cased_pred[:,id]
test_copy=test_copy.drop(columns=['question_title', 'question_body', 'question_user_name',
       'question_user_page', 'answer', 'answer_user_name', 'answer_user_page',
       'url', 'category', 'host'])

In [21]:
postprocessed=postprocess_prediction(test_copy,train)

In [50]:
for column in target_columns:
    if(postprocessed[column]==0.00).all():
        print(f"All values in the column {column} are Zero")

In [ ]:
postprocessed.to_csv("submission.csv", index=False)

In [ ]:
# sub[sub.columns[1:]] = cased_pred
# sub.to_csv("submission.csv", index=False)
# sub.head()